In [1]:
# Step 1: Define Sample Documents
documents = [
    {"section": "Employee Info", "content": "John's pay is processed on the 1st of every month."},
    {"section": "Employee Info", "content": "Mark is on a leave of absence until next Monday."},
    {"section": "Employee Info", "content": "Julie is a software engineer."},
    {"section": "Employee Info", "content": "Julie's pay is processed on the 1st of every month."},
    {"section": "Employee Info", "content": "Mark is a product manager."},
    {"section": "Employee Info", "content": "John is an AI architect and has salary of 500K USD."},
]

# Step 2: Get Content Texts
content_corpus = [doc["content"] for doc in documents]
content_corpus

["John's pay is processed on the 1st of every month.",
 'Mark is on a leave of absence until next Monday.',
 'Julie is a software engineer.',
 "Julie's pay is processed on the 1st of every month.",
 'Mark is a product manager.',
 'John is an AI architect and has salary of 500K USD.']

In [2]:
%pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")
doc_vectors = model.encode(content_corpus)

/Users/suma/Training/11-2025-AItraining/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
doc_vectors
print(doc_vectors.shape)

(6, 384)


In [5]:
# Step 3: User Query and Semantic Matching
import numpy as np

query = "Tell me about John's role."
query_vec = model.encode([query])[0]
query_vec

array([-5.59080578e-02,  3.78226154e-02,  4.02554348e-02, -1.64268352e-02,
       -3.50680649e-02,  1.08953983e-01,  2.83765737e-02,  2.33957507e-02,
       -8.22251365e-02,  2.09271349e-02, -1.45430677e-02,  8.64816904e-02,
        3.82013917e-02,  1.25282630e-02, -4.83173691e-02, -1.05945608e-02,
       -3.96878133e-03,  5.74077293e-02, -3.76466587e-02, -3.85514498e-02,
        1.99908838e-02,  7.95242959e-04,  2.38627996e-02, -4.83551174e-02,
       -8.62931181e-03, -2.50500049e-02,  1.98525246e-02,  5.79859652e-02,
       -5.43468781e-02, -8.26616958e-02, -4.44247685e-02, -1.80384424e-02,
        3.37040275e-02, -2.95444578e-03, -9.83430743e-02,  5.52614853e-02,
        1.23896440e-02,  1.26255974e-02,  1.52711170e-02, -4.67549078e-02,
        1.66643243e-02, -5.56086153e-02,  3.78653891e-02,  3.91717516e-02,
        1.67621579e-02, -9.18189809e-02, -9.79630495e-05, -3.69733311e-02,
       -1.48366829e-02, -3.27884918e-03, -5.06913140e-02,  2.10180245e-02,
        1.48328058e-02, -

In [6]:
%pip install numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [7]:
similarities = model.similarity(query_vec, doc_vectors)

# Ensure it's a 1D numpy array
similarities = np.asarray(similarities).squeeze()

# Now get top 3
top_3_indices = np.argsort(similarities)[::-1][:3]
top_scores = similarities[top_3_indices]

In [8]:
top_scores

array([0.40994582, 0.39578545, 0.2616391 ], dtype=float32)

In [9]:
top_docs = [documents[i]['content'] for i in top_3_indices]

top_docs
context = "\n---\n".join(top_docs)
top_docs, context

(["John's pay is processed on the 1st of every month.",
  'John is an AI architect and has salary of 500K USD.',
  'Mark is a product manager.'],
 "John's pay is processed on the 1st of every month.\n---\nJohn is an AI architect and has salary of 500K USD.\n---\nMark is a product manager.")

In [10]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True, dotenv_path="../.env")
my_api_key = os.getenv("OPEN_AI_API_KEY")

my_client = OpenAI(api_key=my_api_key)
# my_client

def ask_question_open_ai(prompt):

    # print(f"User asked: {prompt}")
    # my_client.chat.completions.create

    llm_response = my_client.chat.completions.create(
        model="gpt-5-nano",
        # messages=[
        #     {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
        #     {"role": "user", "content": prompt}
        # ]
        messages=[
            {"role": "system", "content": '''
             You are an assistant who answers only based on the given context.
             '''},
            {"role": "user", "content": f"Context: {context}\n\n User Question: {query}"} 
        ]

    )
    return llm_response.choices[0].message.content  

In [11]:
print (query)
response = ask_question_open_ai(query)

Tell me about John's role.


In [12]:
print(f"User query: {query}")
print(f"Context: {context}")

print(f"\n\nOpen AI Response: {response}")

User query: Tell me about John's role.
Context: John's pay is processed on the 1st of every month.
---
John is an AI architect and has salary of 500K USD.
---
Mark is a product manager.


Open AI Response: John is an AI architect.
